In [155]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image 
import os, sys
from random import shuffle


IMG_TYPES_COUNT = 10
HIDDEN_LAYER = 40
IMG_SIZE = 28 * 28
MAX_VALUE = 255
plt.style.use('ggplot')
digits = []
for i in range(IMG_TYPES_COUNT): # loading images
    folder_path = f'train/{i}/'
    for dir_ in os.listdir(folder_path): # first 100 digits of all img type
        path = os.path.join(folder_path, dir_)
        with Image.open(path) as img:
            img_arr = np.asarray(img).reshape((IMG_SIZE)) / MAX_VALUE # making each pixel as value in (0, 1)
            digits.append((img_arr, i))
            
shuffle(digits) #for random input
numbers = np.array([i for i, _ in digits])
labels = [i for _, i in digits]
one_hot_labels = np.zeros((len(labels),10))
for i,l in enumerate(labels):
    one_hot_labels[i][l] = 1
labels = one_hot_labels

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os, sys

from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

images, labels = (x_train[0:1000].reshape(1000,28*28) / 255, y_train[0:1000])

one_hot_labels = np.zeros((len(labels),10))
for i,l in enumerate(labels):
    one_hot_labels[i][l] = 1
labels = one_hot_labels

test_images = x_test.reshape(len(x_test),28*28) / 255
test_labels = np.zeros((len(y_test),10))
for i,l in enumerate(y_test):
    test_labels[i][l] = 1

2024-03-04 21:10:18.590607: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 21:10:18.590943: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 21:10:18.717884: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-04 21:10:21.647483: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [38]:
relu = lambda x: (x > 0) * x
relu2deriv = lambda x: x > 0

tanh = lambda x: np.tanh(x)
tanh2deriv = lambda x: 1 - x ** 2

def softmax(x):
    temp = np.exp(x)
    return temp / np.sum(temp, axis=1, keepdims=True)

def get_image_section(layer, row_from, row_to, col_from, col_to):
    section = layer[:, row_from:row_to, col_from:col_to]
    return section.reshape(-1, 1, row_to - row_from, col_to - col_from)

def get_pred_from_test(path='test.png'): # I use Aseprite for drawing
    with Image.open(path) as img:
        img.load()
        img_arr = np.asarray(img).reshape(1, 28, 28) / 255

    layer_0 = img_arr
    layer_0 = layer_0.reshape(-1, 28, 28)

    sects = []
    for row_start in range(layer_0.shape[1] - kernel_rows):
        for col_start in range(layer_0.shape[2] - kernel_cols):
            sect = get_image_section(layer_0,
                                     row_start,
                                     row_start + kernel_rows,
                                     col_start,
                                     col_start + kernel_cols)

            sects.append(sect)

    expanded_input = np.concatenate(sects, axis=1)
    es = expanded_input.shape
    flattened_input = expanded_input.reshape(es[0] * es[1], -1)

    kernel_output = flattened_input.dot(kernels)
    layer_1 = tanh(kernel_output.reshape(es[0],-1))
    dropout_mask = np.random.randint(2,size=layer_1.shape)
    layer_1 *= dropout_mask * 2
    layer_2 = softmax(np.dot(layer_1,weights_1_2))
    
    print(layer_2.argmax())
    return img, layer_2

In [65]:
batch_size = 128
alpha, iterations = (2, 300)
pixels_per_image, num_labels = (784, 10)

input_rows = 28
input_cols = 28

kernel_rows = 2
kernel_cols = 2
num_kernels = 20

hidden_size = ((input_rows - kernel_rows) *
               (input_cols - kernel_cols)) * num_kernels

kernels = 0.02 * np.random.random((kernel_cols * kernel_rows, num_kernels)) - 0.01

weights_1_2 = 0.2 * np.random.random((hidden_size, num_labels)) - 0.1

for j in range(iterations):
    error, correct_cnt = (0.0, 0)
    for i in range(int(len(images) / batch_size)):
        batch_start, batch_end = ((i * batch_size),((i+1)*batch_size))
        layer_0 = images[batch_start:batch_end]
        layer_0 = layer_0.reshape(-1, 28, 28)
        
        sects = []
        for row_start in range(layer_0.shape[1] - kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(layer_0,
                                         row_start,
                                         row_start + kernel_rows,
                                         col_start,
                                         col_start + kernel_cols)
                
                sects.append(sect) 

        expanded_input = np.concatenate(sects, axis=1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0] * es[1], -1)

        kernel_output = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_output.reshape(es[0],-1))
        dropout_mask = np.random.randint(2,size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = softmax(np.dot(layer_1,weights_1_2))

        error += np.sum((labels[batch_start:batch_end] - layer_2) ** 2)
        for k in range(batch_size):
            labelset = labels[batch_start+k:batch_start+k+1]
            _inc = int(np.argmax(layer_2[k:k+1]) ==
                       np.argmax(labelset))
            correct_cnt += _inc

        layer_2_delta = (labels[batch_start:batch_end]-layer_2) / (batch_size * layer_2.shape[0])
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * tanh2deriv(layer_1)
        layer_1_delta *= dropout_mask

        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        l1d_reshape = layer_1_delta.reshape(kernel_output.shape)
        k_update = flattened_input.T.dot(l1d_reshape)
        kernels -= alpha * k_update

    if j % 1 == 0:
        sys.stdout.write("\r" + \
                         "I:" + str(j) + \
                         " Train-Err:" + str(error/ float(len(images)))[0:5] + \
                         " Train-Acc:" + str(correct_cnt/ float(len(images))))

I:299 Train-Err:0.141 Train-Acc:0.798

In [11]:
layer_2.shape

(100, 10)

In [77]:
get_pred_from_test()[0]

7


In [79]:
|import numpy as np, sys
np.random.seed(1)

from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

images, labels = (x_train[0:1000].reshape(1000,28*28) / 255, y_train[0:1000])

one_hot_labels = np.zeros((len(labels),10))
for i,l in enumerate(labels):
    one_hot_labels[i][l] = 1
labels = one_hot_labels

test_images = x_test.reshape(len(x_test),28*28) / 255
test_labels = np.zeros((len(y_test),10))
for i,l in enumerate(y_test):
    test_labels[i][l] = 1

def tanh(x):
    return np.tanh(x)

def tanh2deriv(output):
    return 1 - (output ** 2)

def softmax(x):
    temp = np.exp(x)
    return temp / np.sum(temp, axis=1, keepdims=True)

alpha, iterations, hidden_size = (2, 300, 100)
pixels_per_image, num_labels = (784, 10)
batch_size = 100

weights_0_1 = 0.02*np.random.random((pixels_per_image,hidden_size))-0.01
weights_1_2 = 0.2*np.random.random((hidden_size,num_labels)) - 0.1

for j in range(iterations):
    correct_cnt = 0
    for i in range(int(len(images) / batch_size)):
        batch_start, batch_end=((i * batch_size),((i+1)*batch_size))
        layer_0 = images[batch_start:batch_end]
        layer_1 = tanh(np.dot(layer_0,weights_0_1))
        dropout_mask = np.random.randint(2,size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = softmax(np.dot(layer_1,weights_1_2))

        for k in range(batch_size):
            correct_cnt += int(np.argmax(layer_2[k:k+1]) == np.argmax(labels[batch_start+k:batch_start+k+1]))

        layer_2_delta = (labels[batch_start:batch_end]-layer_2) / (batch_size * layer_2.shape[1])
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * tanh2deriv(layer_1)
        layer_1_delta *= dropout_mask

        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)

    test_correct_cnt = 0
    if(j % 10 == 0):
        sys.stdout.write("\n"+ \
                         "I:" + str(j) + \
                         " Test-Acc:"+str(test_correct_cnt/float(len(test_images)))+ \
                         " Train-Acc:" + str(correct_cnt/float(len(images))))
        


I:0 Test-Acc:0.0 Train-Acc:0.48
I:10 Test-Acc:0.0 Train-Acc:0.88
I:20 Test-Acc:0.0 Train-Acc:0.919
I:30 Test-Acc:0.0 Train-Acc:0.951
I:40 Test-Acc:0.0 Train-Acc:0.944
I:50 Test-Acc:0.0 Train-Acc:0.957
I:60 Test-Acc:0.0 Train-Acc:0.948
I:70 Test-Acc:0.0 Train-Acc:0.952
I:80 Test-Acc:0.0 Train-Acc:0.941
I:90 Test-Acc:0.0 Train-Acc:0.948

KeyboardInterrupt: 